# Lifespan Synthesis

### Synthesis one's face with different ages and render into a video.

Original Author: [royorel](https://github.com/royorel)

Colab Provided by: [BreezeWhite](https://github.com/BreezeWhite)

Original Github: https://github.com/royorel/Lifespan_Age_Transformation_Synthesis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BreezeWhite/interesting-colabs/blob/main/scripts/LifespanSynthesis.ipynb)

In [ ]:
#@title Setup

!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install unidecode dominate

%run download_models.py


import os
from collections import OrderedDict
from IPython.display import HTML
from base64 import b64encode

import util.util as util
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
from util.visualizer import Visualizer


opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)


def gen_video_html(img_path, fine_size):
  webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
  !ffmpeg -i $out_path -c:v libvpx-vp9 -crf 30 -b:v 0 -b:a 128k -c:a libopus $webm_out_path

  video_type = "video/webm"
  video = open(webm_out_path, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(video).decode()
  return HTML(f"""
    <video width={fine_size} controls>
      <source src="{data_url}" type="{video_type}">
    </video>
  """)


In [ ]:
#@title Upload Image

from google.colab import files
from IPython.display import display
from PIL import Image

ff = files.upload()
f_name = list(ff.keys())[0]
display(Image.open(f_name))

In [ ]:
#@title Run

from IPython.display import clear_output

is_man = False #@param {type:"boolean"}

opt.name = 'males_model' if is_man else 'females_model'
model = create_model(opt)
model.eval()
clear_output()

data = dataset.dataset.get_item_from_path(f_name)
visuals = model.inference(data)
os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(f_name)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

html = gen_video_html(f_name, opt.fineSize)
clear_output()
display(html)
